In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


In [0]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
df = spark \
.readStream \
.format('kinesis') \
.option('streamName','Kinesis-Prod-Stream') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

# Define geo structure
geo_struct = StructType([
    StructField("ind", IntegerType(),True),
    StructField("timestamp", StringType(),True),
    StructField("latitude", StringType(),True),
    StructField("longitude", StringType(),True),
    StructField("country", StringType(),True)                        
])

df_geo = df.filter(df.partitionKey == "streaming-57e94de2a910-geo")
# Decode JSON Data
df_geo = df_geo.select(from_json(col("data").cast("string"), geo_struct).alias("parsed_value")).select("parsed_value.*")
#df_geo = df_geo.selectExpr("CAST(data as STRING) jsonData")
#df_geo = df_geo.select(from_json("jsonData", geo_struct))

display(df_geo)


ind,timestamp,latitude,longitude,country
7528,2020-08-28T03:52:47,-89.9787,-173.293,Albania
7528,2020-08-28T03:52:47,-89.9787,-173.293,Albania
2863,2020-04-27T13:34:16,-5.34445,-177.924,Armenia
5730,2021-04-19T17:37:03,-77.015,-101.437,Colombia
7528,2020-08-28T03:52:47,-89.9787,-173.293,Albania
2863,2020-04-27T13:34:16,-5.34445,-177.924,Armenia
5730,2021-04-19T17:37:03,-77.015,-101.437,Colombia
8304,2019-09-13T04:50:29,-28.8852,-164.87,French Guiana
8731,2020-07-17T04:39:09,-83.104,-171.302,Aruba
1313,2018-06-26T02:39:25,77.0447,61.9119,Maldives


In [0]:
# Define pin structure
pin_struct = StructType([
    StructField("index", IntegerType(),True),
    StructField("unique_id", StringType(),True),
    StructField("title", StringType(),True),
    StructField("description", StringType(),True),
    StructField("poster_name", StringType(),True),
    StructField("follower_count", StringType(),True),
    StructField("tag_list", StringType(),True),
    StructField("is_image_or_video", StringType(),True),
    StructField("image_src", StringType(),True),
    StructField("downloaded", StringType(),True),
    StructField("save_location", StringType(),True),
    StructField("category", StringType(),True)
])
df_pin = df.filter(df.partitionKey == "streaming-57e94de2a910-pin")
# Decode JSON Data
df_pin = df_pin.select(from_json(col("data").cast("string"), pin_struct).alias("parsed_value")).select("parsed_value.*")
#df_pin = df_pin.selectExpr("CAST(data as STRING) jsonData")

display(df_pin)


index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,1,Local save in /data/finance,finance
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,1,Local save in /data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,Commitment Connection,51k,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,1,Local save in /data/qu

In [0]:
# Define user structure
user_struct = StructType([
    StructField("ind", IntegerType(),True),
    StructField("first_name", StringType(),True),
    StructField("last_name", StringType(),True),
    StructField("age", IntegerType(),True),
    StructField("date_joined", StringType(),True)
])
df_user = df.filter(df.partitionKey == "streaming-57e94de2a910-user")
# Decode JSON Data
df_user = df_user.select(from_json(col("data").cast("string"), user_struct).alias("parsed_value")).select("parsed_value.*")
#df_user = df_user.selectExpr("CAST(data as STRING) jsonData")
#df_user = df_user.select(from_json("jsonData", user_struct))

display(df_user)

ind,first_name,last_name,age,date_joined
7528,Abigail,Ali,20,2015-10-24T11:23:51
7528,Abigail,Ali,20,2015-10-24T11:23:51
7528,Abigail,Ali,20,2015-10-24T11:23:51
2863,Dylan,Holmes,32,2016-10-23T14:06:51
5730,Rachel,Davis,36,2015-12-08T20:02:43
7528,Abigail,Ali,20,2015-10-24T11:23:51
2863,Dylan,Holmes,32,2016-10-23T14:06:51
5730,Rachel,Davis,36,2015-12-08T20:02:43
8304,Charles,Berry,25,2015-12-28T04:21:39
8731,Andrea,Alexander,21,2015-11-10T09:27:42


In [0]:
# Save as a Delta table in Databricks in append mode
table_name = "57e94de2a910_geo_kinesis"  # e.g., "57e94de2a910_geo_kinesis"
df_geo.writeStream \
    .format("delta")\
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/geo/") \
    .table(table_name)

print(f"Data from streaming-57e94de2a910-geo saved as table {table_name}")

Data from streaming-57e94de2a910-geo saved as table 57e94de2a910_geo_kinesis


In [0]:
# Save as a Delta table in Databricks in append mode
table_name = "57e94de2a910_pin_kinesis"  # e.g., "57e94de2a910_pin_kinesis"
df_pin.writeStream \
    .format("delta")\
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/pin/") \
    .table(table_name)

print(f"Data from streaming-57e94de2a910-pin saved as table {table_name}")


Data from streaming-57e94de2a910-pin saved as table 57e94de2a910_pin_kinesis


In [0]:
# Save as a Delta table in Databricks in append mode
table_name = "57e94de2a910_user_kinesis"  # e.g., "57e94de2a910_user_kinesis"
df_user.writeStream \
    .format("delta")\
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/user/") \
    .table(table_name)

print(f"Data from streaming-57e94de2a910-user saved as table {table_name}")

Data from streaming-57e94de2a910-user saved as table 57e94de2a910_user_kinesis


In [0]:
%sql
SELECT * FROM 57e94de2a910_pin_kinesis LIMIT 10;


index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,1,Local save in /data/finance,finance
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,1,Local save in /data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,Commitment Connection,51k,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,1,Local save in /data/qu